# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head(20)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [2]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
PassengerId 乘客編號
Survived 是否倖存
Pclass 船票等級--------類別
Name 乘客姓名
Sex 乘客性別
Age 年齡--------數值
SibSp 堂兄弟妹
Parch 父母與小孩
Ticket 船票號碼
Fare 船票價格
Cabin 船艙
Embarked 登入港口

for item in list(df.columns):
    print(df[item].unique(),'\n')
    
Pclass:[3 1 2]
Age:[22.   38.   26.   35.     nan 54.    2.   27.   14.    4.   58.   20.
 39.   55.   31.   34.   15.   28.    8.   19.   40.   66.   42.   21.
 18.    3.    7.   49.   29.   65.   28.5   5.   11.   45.   17.   32.
 16.   25.    0.83 30.   33.   23.   24.   46.   59.   71.   37.   47.
 14.5  70.5  32.5  12.    9.   36.5  51.   55.5  40.5  44.    1.   61.
 56.   50.   36.   45.5  20.5  62.   41.   52.   63.   23.5   0.92 43.
 60.   10.   64.   13.   48.    0.75 53.   57.   80.   70.   24.5   6.
  0.67 30.5   0.42 34.5  74.  ]
"""
df['Age']=df['Age'].fillna(df['Age'].mean())
#==========用統計值作為特徵==========
df_mean=df.groupby(['Pclass'])['Age'].mean()
df_mean=df_mean.reset_index()

df_mode = df.groupby(['Pclass'])['Age'].apply(lambda x: x.mode()[0]).reset_index()

print(df_mean,'\n',df_mode)
#==========用統計值作為特徵==========

temp=pd.merge(df_mean,df_mode,how='left',on=['Pclass'])
temp.columns=['Pclass','Mean_age','Median_age']
temp

   Pclass        Age
0       1  37.048118
1       2  29.866958
2       3  26.403259 
    Pclass        Age
0       1  29.699118
1       2  29.699118
2       3  29.699118


,Pclass,Mean_age,Median_age
0,1,37.048118,29.699118
1,2,29.866958,29.699118
2,3,26.403259,29.699118


In [3]:
df = pd.merge(df, temp, how='left', on=['Pclass'])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Mean_age,Median_age
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,26.403259,29.699118
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,37.048118,29.699118
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,26.403259,29.699118
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,37.048118,29.699118
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,26.403259,29.699118


In [4]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

7 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Mean_age', 'Median_age']



,Pclass,Age,SibSp,Parch,Fare,Mean_age,Median_age
0,3,22.0,1,0,7.2500,26.403259,29.699118
1,1,38.0,1,0,71.2833,37.048118,29.699118
2,3,26.0,0,0,7.9250,26.403259,29.699118
3,1,35.0,1,0,53.1000,37.048118,29.699118
4,3,35.0,0,0,8.0500,26.403259,29.699118


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [7]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_origin = df.drop(['Mean_age', 'Median_age'] , axis=1)
    #只含原始特徵
train_X = MMEncoder.fit_transform(df_origin)
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6982266036406296

In [9]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
    #包含新特徵
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6971093561322185

## 使用Fare/Age再做一次

In [10]:
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head(20)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [11]:
df['Fare'].unique()

array([  7.25  ,  71.2833,   7.925 ,  53.1   ,   8.05  ,   8.4583,
        51.8625,  21.075 ,  11.1333,  30.0708,  16.7   ,  26.55  ,
        31.275 ,   7.8542,  16.    ,  29.125 ,  13.    ,  18.    ,
         7.225 ,  26.    ,   8.0292,  35.5   ,  31.3875, 263.    ,
         7.8792,   7.8958,  27.7208, 146.5208,   7.75  ,  10.5   ,
        82.1708,  52.    ,   7.2292,  11.2417,   9.475 ,  21.    ,
        41.5792,  15.5   ,  21.6792,  17.8   ,  39.6875,   7.8   ,
        76.7292,  61.9792,  27.75  ,  46.9   ,  80.    ,  83.475 ,
        27.9   ,  15.2458,   8.1583,   8.6625,  73.5   ,  14.4542,
        56.4958,   7.65  ,  29.    ,  12.475 ,   9.    ,   9.5   ,
         7.7875,  47.1   ,  15.85  ,  34.375 ,  61.175 ,  20.575 ,
        34.6542,  63.3583,  23.    ,  77.2875,   8.6542,   7.775 ,
        24.15  ,   9.825 ,  14.4583, 247.5208,   7.1417,  22.3583,
         6.975 ,   7.05  ,  14.5   ,  15.0458,  26.2833,   9.2167,
        79.2   ,   6.75  ,  11.5   ,  36.75  ,   7.7958,  12.5

In [12]:
df['Age']=df['Age'].fillna(df['Age'].mean())
#==========用統計值作為特徵==========
df_mean=df.groupby(['Fare'])['Age'].mean()
df_mean=df_mean.reset_index()

df_mode = df.groupby(['Fare'])['Age'].apply(lambda x: x.mode()[0]).reset_index()

print(df_mean,'\n',df_mode)
#==========用統計值作為特徵==========

temp=pd.merge(df_mean,df_mode,how='left',on=['Fare'])
temp.columns=['Fare','Mean_age','Median_age']
temp

         Fare        Age
0      0.0000  32.239529
1      4.0125  20.000000
2      5.0000  33.000000
3      6.2375  61.000000
4      6.4375  34.500000
5      6.4500  43.000000
6      6.4958  26.000000
7      6.7500  18.500000
8      6.8583  29.699118
9      6.9500  29.699118
10     6.9750  36.000000
11     7.0458  29.000000
12     7.0500  28.099874
13     7.0542  34.000000
14     7.1250  28.750000
15     7.1417  24.000000
16     7.2250  29.641225
17     7.2292  25.126255
18     7.2500  29.199796
19     7.3125  29.699118
20     7.4958  26.000000
21     7.5208  22.000000
22     7.5500  31.099559
23     7.6292  29.699118
24     7.6500  26.750000
25     7.7250  29.699118
26     7.7292  29.699118
27     7.7333  24.099559
28     7.7375  29.699118
29     7.7417  25.000000
..        ...        ...
218   80.0000  50.000000
219   81.8583   4.000000
220   82.1708  28.849559
221   83.1583  39.666667
222   83.4750  40.000000
223   86.5000  26.333333
224   89.1042  39.349559
225   90.0000  37.500000


,Fare,Mean_age,Median_age
0,0.0000,32.239529,29.699118
1,4.0125,20.000000,20.000000
2,5.0000,33.000000,33.000000
3,6.2375,61.000000,61.000000
4,6.4375,34.500000,34.500000
5,6.4500,43.000000,43.000000
6,6.4958,26.000000,18.000000
7,6.7500,18.500000,18.000000
8,6.8583,29.699118,29.699118
9,6.9500,29.699118,29.699118


In [13]:
df = pd.merge(df, temp, how='left', on=['Fare'])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Mean_age,Median_age
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,29.199796,22.000000
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,38.000000,38.000000
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,28.555556,32.000000
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,30.200000,19.000000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,30.378782,29.699118


In [14]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

7 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Mean_age', 'Median_age']



,Pclass,Age,SibSp,Parch,Fare,Mean_age,Median_age
0,3,22.0,1,0,7.2500,29.199796,22.000000
1,1,38.0,1,0,71.2833,38.000000,38.000000
2,3,26.0,0,0,7.9250,28.555556,32.000000
3,1,35.0,1,0,53.1000,30.200000,19.000000
4,3,35.0,0,0,8.0500,30.378782,29.699118


In [15]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_origin = df.drop(['Mean_age', 'Median_age'] , axis=1)
    #只含原始特徵
train_X = MMEncoder.fit_transform(df_origin)
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6982266036406296

In [16]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
    #包含新特徵
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7060602449759502